In [1]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datetime import datetime
import os
import json

In [2]:
# ✅ Detect MPS
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: mps


In [3]:
project_root = os.path.abspath("..")

In [4]:
# Load dataset
dataset_version = {
    "joined": os.path.join(project_root, "datasets/processed_joined_toxicity_data.csv"),
    "split": os.path.join(project_root, "datasets/processed_split_toxicity_data.csv"),
    "message_only": os.path.join(project_root, "datasets/processed_split_toxicity_data.csv")
}

# Three tokenizers
def tokenize_joined(example):
    return tokenizer(example["input_text"], padding="max_length", truncation=True, max_length=512)

def tokenize_split(example):
    return tokenizer(example["context"], example["message"], padding="max_length", truncation=True, max_length=512)

def tokenize_message_only(example):
    return tokenizer(example["message"], padding="max_length", truncation=True, max_length=512)

tokenize_funcs = {
    "joined": tokenize_joined,
    "split": tokenize_split,
    "message_only": tokenize_message_only
}

In [ ]:
BEST_MODELS_PATH = "best_models.json"
F1_TOLERANCE = 1e-2

def load_best_models():
    if not os.path.exists(BEST_MODELS_PATH):
        return {
            'bert-base-uncased-joined': {'f1': 0.0, 'precision': 0.0},
            'bert-base-uncased-split': {'f1': 0.0, 'precision': 0.0},
            'bert-base-uncased-message_only': {'f1': 0.0, 'precision': 0.0},
            'bert-large-uncased-joined': {'f1': 0.0, 'precision': 0.0},
            'bert-large-uncased-split': {'f1': 0.0, 'precision': 0.0},
            'bert-large-uncased-message_only': {'f1': 0.0, 'precision': 0.0},
            's-nlp-roberta-toxicity-classifier-joined': {'f1': 0.0, 'precision': 0.0},
            's-nlp-roberta-toxicity-classifier-split': {'f1': 0.0, 'precision': 0.0},
            's-nlp-roberta-toxicity-classifier-message_only': {'f1': 0.0, 'precision': 0.0}
        }
    with open(BEST_MODELS_PATH, "r") as f:
        return json.load(f)

In [6]:
def update_best_model(model_name, f1_score, precision):
    best_models = load_best_models()
    current_best = best_models.get(model_name, {"f1": 0.0, "precision": 0.0})

    better_f1 = f1_score - F1_TOLERANCE > current_best["f1"]
    similar_f1 = abs(f1_score - current_best["f1"]) <= F1_TOLERANCE
    better_precision = precision > current_best["precision"]

    should_update = better_f1 or (similar_f1 and better_precision)

    if should_update:
        print(f"🎯 New best for {model_name}!")
        print(f"F1: {f1_score:.4f} (prev: {current_best['f1']:.4f}) | Precision: {precision:.4f} (prev: {current_best['precision']:.4f})")
        best_models[model_name] = {
            "f1": f1_score,
            "precision": precision,
        }
        with open(BEST_MODELS_PATH, "w") as f:
            json.dump(best_models, f, indent=2)
        return True
    else:
        print(f"🧪 {model_name} did not improve:")
        print(f"F1: {f1_score:.4f} (best: {current_best['f1']:.4f}) | Precision: {precision:.4f} (best: {current_best['precision']:.4f})")
        return False

In [ ]:
results = {}
best_f1 = -1
best_model = None
best_version = ""

# Load tokenizer
tokenizer = RobertaTokenizer.from_pretrained("s-nlp/roberta_toxicity_classifier")

for version, tokenizer_func in tokenize_funcs.items():
    print(f"\nTraining version: {version}")
    df = pd.read_csv(dataset_version[version])  # Must contain 'input_text', 'context', 'message', 'label'
    dataset = Dataset.from_pandas(df)
    tokenized = dataset.map(tokenizer_func, batched=True)
    tokenized = tokenized.train_test_split(test_size=0.25)
    tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

    model_name = f"s-nlp-roberta-toxicity-classifier-{version}"
    model = RobertaForSequenceClassification.from_pretrained("s-nlp/roberta_toxicity_classifier", num_labels=2)
    model.to(device)

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=-1)
        acc = accuracy_score(labels, preds)
        precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary', zero_division=0)
        return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

    training_args = TrainingArguments(
        output_dir="./results",
        eval_strategy="epoch",             # Evaluate every epoch
        save_strategy="epoch",                   # Save every epoch
        learning_rate=2e-5,                      # From table
        per_device_train_batch_size=16,          # From table
        per_device_eval_batch_size=16,           # From table
        num_train_epochs=5,                      # From table
        weight_decay=0.01,                       # From table
        warmup_ratio=0.0,                        # Table didn't specify, assume 0 unless specified
        lr_scheduler_type="linear",              # From table
        logging_dir="./logs",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        optim="adamw_torch",                     # Explicitly using AdamW
        adam_epsilon=1e-8                        # From table
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized["train"],
        eval_dataset=tokenized["test"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    eval_result = trainer.evaluate()
    results[version] = eval_result

    if update_best_model(
        model_name=model_name,
        f1_score=eval_result["eval_f1"],
        precision=eval_result["eval_precision"],
    ):
        trainer.save_model(f"models/best-{model_name}")

    if eval_result["eval_f1"] > best_f1:
        best_f1 = eval_result["eval_f1"]
        best_model = model
        best_version = version

# Print summary
print("\n--- Summary of Results ---")
for version, result in results.items():
    print(f"{version.upper()}: F1 = {result['eval_f1']:.4f}")

print(f"\nBest model: {best_version.upper()} (F1 = {best_f1:.4f})")